In [ ]:
# hide
# dont_test
%load_ext nb_black

<IPython.core.display.Javascript object>

# Create an Example Django Project

## Use django-admin to bootstrap the project

In [ ]:
# dont_test

%cd ..

/Users/jochen/projects/django_fileresponse


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

!django-admin startproject example

## Views

In [ ]:
%%writefile example/example/views.py
# dont_test

import aiofiles

from pathlib import Path

from django.conf import settings
from django.http import HttpResponse
from django.http import FileResponse

from fileresponse.http import AiofileFileResponse
from fileresponse.http import AiobotocoreFileResponse


def get_file(request, num=None):
    file_path = Path(__file__).parent.parent / "data" / str(num)
    print(str(file_path))
    return FileResponse(file_path.open("rb"))


async def aget_file(request, num=None):
    file_path = Path(__file__).parent.parent / "data" / str(num)
    return AiofileFileResponse(file_path, chunk_size=4096)


async def aminio_get_file(request, num=None):
    bucket = settings.DJANGO_AWS_STORAGE_BUCKET_NAME
    key = str(num)
    return AiobotocoreFileResponse(bucket, key, chunk_size=4096)

Overwriting example/example/views.py


<IPython.core.display.Javascript object>

## Urls

In [ ]:
%%writefile example/example/urls.py
# dont_test

from django.urls import path

from . import views

urlpatterns = [
    path("sync/<int:num>", views.get_file),
    path("async_filesystem/<int:num>", views.aget_file),
    path("async_minio/<int:num>", views.aminio_get_file),
]

Overwriting example/example/urls.py


<IPython.core.display.Javascript object>

## ASGI

In [ ]:
%%writefile example/example/asgi.py
# dont_test
import os
import django

from fileresponse.asgi import get_asgi_application

from . import urls

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'example.settings')
application = get_asgi_application()

Overwriting example/example/asgi.py


<IPython.core.display.Javascript object>

## Settings

In [ ]:
%%writefile example/example/settings.py
# dont_test


from pathlib import Path

# Build paths inside the project like this: BASE_DIR / "subdir".
BASE_DIR = Path(__file__).resolve().parent.parent


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/3.2/howto/deployment/checklist/

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = "django-insecure-eyj(*+5=#xb1(5$50p*k0%5evl)80dcoh(i6(tx9a7dq9i5#9f"

# SECURITY WARNING: don"t run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]

ROOT_URLCONF = "example.urls"

TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [],
        "APP_DIRS": True,
        "OPTIONS": {
            "context_processors": [
                "django.template.context_processors.debug",
                "django.template.context_processors.request",
                "django.contrib.auth.context_processors.auth",
                "django.contrib.messages.context_processors.messages",
            ],
        },
    },
]

WSGI_APPLICATION = "example.wsgi.application"


# Database
# https://docs.djangoproject.com/en/3.2/ref/settings/#databases

DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.sqlite3",
        "NAME": BASE_DIR / "db.sqlite3",
    }
}


# Password validation
# https://docs.djangoproject.com/en/3.2/ref/settings/#auth-password-validators

AUTH_PASSWORD_VALIDATORS = [
    {
        "NAME": "django.contrib.auth.password_validation.UserAttributeSimilarityValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.MinimumLengthValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.CommonPasswordValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.NumericPasswordValidator",
    },
]


# Internationalization
# https://docs.djangoproject.com/en/3.2/topics/i18n/

LANGUAGE_CODE = "en-us"

TIME_ZONE = "UTC"

USE_I18N = True

USE_L10N = True

USE_TZ = True


# Static files (CSS, JavaScript, Images)
# https://docs.djangoproject.com/en/3.2/howto/static-files/

STATIC_URL = "/static/"

# Default primary key field type
# https://docs.djangoproject.com/en/3.2/ref/settings/#default-auto-field

DEFAULT_AUTO_FIELD = "django.db.models.BigAutoField"

# AWS S3 / MinIO settings
FILERESPONSE_S3_ACCESS_KEY_ID="minioadmin"
FILERESPONSE_S3_SECRET_ACCESS_KEY="minioadmin"
FILERESPONSE_S3_REGION="us-west-2"
FILERESPONSE_S3_STORAGE_BUCKET_NAME="fileresponse"
FILERESPONSE_S3_ENDPOINT_URL="http://localhost:9000"

Overwriting example/example/settings.py


<IPython.core.display.Javascript object>

## Apply Migrations

In [ ]:
# dont_test

%cd example
!python manage.py migrate

/Users/jochen/projects/django_fileresponse/example
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... 

## Link example to Notebooks Directory

In [ ]:
# dont_test

%cd ../nbs
!ln -s ../example/example .
%cd -

/Users/jochen/projects/django_fileresponse/nbs
ln: ./example: File exists
/Users/jochen/projects/django_fileresponse/example


## Start Server

Just run:
```shell
python manage.py runserver 0.0.0.0:8000
```

In [ ]:
# dont_test

!python manage.py runserver 0.0.0.0:8000

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
June 21, 2021 - 08:13:00
Django version 3.2.3, using settings 'example.settings'
Starting development server at http://0.0.0.0:8000/
Quit the server with CONTROL-C.
/Users/jochen/projects/django_fileresponse/example/data/0
[21/Jun/2021 08:13:25] "GET /0 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/1
[21/Jun/2021 08:13:25] "GET /1 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/2
[21/Jun/2021 08:13:25] "GET /2 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/3
[21/Jun/2021 08:13:25] "GET /3 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/4
[21/Jun/2021 08:13:25] "GET /4 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/5
[21/Jun/2021 08:13:25] "GET /5 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse

/Users/jochen/projects/django_fileresponse/example/data/67
[21/Jun/2021 08:13:28] "GET /67 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/68
[21/Jun/2021 08:13:28] "GET /68 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/69
[21/Jun/2021 08:13:28] "GET /69 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/70
[21/Jun/2021 08:13:28] "GET /70 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/71
[21/Jun/2021 08:13:28] "GET /71 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/72
[21/Jun/2021 08:13:28] "GET /72 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/73
[21/Jun/2021 08:13:28] "GET /73 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/74
[21/Jun/2021 08:13:28] "GET /74 HTTP/1.1" 200 10485760
/Users/jochen/projects/django_fileresponse/example/data/75
[21/Jun/2021 08:13:28] "GET /